In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from splits import split_users # contains split_users func

pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 100)

In [2]:
# # read in pickled DFs generated by query_dfs.py

# df_orders = pd.read_pickle("./pickle/df_orders.pickle")
# df_train = pd.read_pickle("./pickle/df_train.pickle")
# df_prior = pd.read_pickle("./pickle/df_prior.pickle")
# df_prod_detail = pd.read_pickle("./pickle/df_prod_detail.pickle")

In [3]:
# read in pickled feature DF generated by feature_engineering_1.ipynb
X = pd.read_pickle("./pickle/X_25.pickle")

In [4]:
# check scikit-learn version
import sklearn
print(sklearn.__version__)

0.24.1


In [5]:
# X_train, X_test, y_train, y_test = split_users(X, subset=0.5, test_size=0.2)# only using 10%


In [6]:


# rf = RandomForestClassifier(n_estimators=2000, max_depth=8, max_features=8, n_jobs=-1)
# rf_fit = rf.fit(X_train, y_train)


In [7]:
# preds = rf_fit.predict(X_test)

In [8]:
# f1_score(y_test, preds)

In [9]:
# with open(f"models/rf_fit.pickle", "wb") as pfile:
#         pickle.dump(rf_fit, pfile)

In [10]:


# X_train, X_test, y_train, y_test = split_users(X, subset=0.2, test_size=0.2) # only using 10%

# xgb = XGBClassifier(n_estimators=300, eval_metric='logloss', use_label_encoder =False)
# xgb_fit = xgb.fit(X_train, y_train)

In [11]:
# with open(f"models/xgboost_fit.pickle", "wb") as pfile:
#         pickle.dump(xgb_fit, pfile)

In [12]:
# xgb_fit.score(X_test, y_test)

In [13]:
# xgb_preds = xgb_fit.predict(X_test)

In [14]:
# f1_score(y_test, xgb_preds)

## Grid Search

In [15]:
# X_train, X_test, y_train, y_test = split_users(X, subset=0.25, test_size=0.2)

In [16]:
# estimator = XGBClassifier(objective='binary:logistic',
#                           use_label_encoder=False,
#                           eval_metric='logloss',
#                           random_state=54,
# )

# params = {
#     'max_depth': range(5, 12, 1),
#     'n_estimators': [475, 500, 525],
#     'learning_rate': [0.009, 0.01, 0.011]
# }

In [17]:
# grid_search = GridSearchCV(
#     estimator = estimator,
#     param_grid = params,
#     verbose=10
# )

In [18]:
# %%time
# grid_xgb = grid_search.fit(X_train, y_train)
# print("The best parameters are: \n", grid_search.best_params_)



In [19]:
# with open(f"models/grid_xgb.pickle", "wb") as pfile:
#         pickle.dump(grid_xgb, pfile)

# ELLIOT - SATURDAY MORNING

### Grid Search Results

**On a different VM, I used grid search to tune min_child_weight and colsample_bytree parameters. Here was the grid & results:**

```
estimator = XGBClassifier(objective='binary:logistic',
                          use_label_encoder=False,
                          eval_metric='logloss',
                          random_state=54,
                          max_depth=3,
                          learning_rate=0.01,
                          n_estimators=500
)

params = {
    'min_child_weight': range(1, 10, 1),
    'colsample_bytree': [.6, .7, .8, .9, 1.0]
}

# results
Fitting 5 folds for each of 45 candidates, totalling 225 fits
The best parameters are: 
 {'colsample_bytree': 0.7, 'min_child_weight': 8}
CPU times: user 1h 47min 29s, sys: 1.55 s, total: 1h 47min 30s
Wall time: 10min 54s

```

**Below were the parameters for our grid search.**
```
X_train, X_test, y_train, y_test = split_users(X, subset=0.25, test_size=0.2)

estimator = XGBClassifier(objective='binary:logistic',
                          use_label_encoder=False,
                          eval_metric='logloss',
                          random_state=54,
)

params = {
    'max_depth': range(5, 12, 1),
    'n_estimators': [475, 500, 525],
    'learning_rate': [0.009, 0.01, 0.011]
    
grid_search = GridSearchCV(
estimator = estimator,
param_grid = params
)
}
```

**And the results:**

```
The best parameters are: 
 {'learning_rate': 0.009, 'max_depth': 8, 'n_estimators': 475}
CPU times: user 16d 5h 11min 23s, sys: 5min 59s, total: 16d 5h 17min 23s
Wall time: 1d 3h 17min 53s
```

Great. Now we have our parameters. Let's run the model!

One with `colsample_bytree` & `min_child_weight` results from the grid search done in another workbook (changes were not committed).
- the above grid parameters were not included, so they've never all been used together. 

So we'll actually run two models....one with, and one without those two.

In [28]:
X['streak'] = X['streak'].astype(float)

In [29]:
X_train, X_test, y_train, y_test = split_users(X, subset=False, test_size=0.2)


    X_train sample size: 6782401
    X_test sample size: 1692260


In [30]:
X.dtypes

product_id                  int64
user_id                     int64
user_total_prod_orders      int64
cart                       object
in_cart                     int64
last_cart                  object
in_last_cart                int64
prod_prior_sales            int64
prod_pct_reorders         float64
prod_avg_atco             float64
user_avg_ippo             float64
days_since_prior_order    float64
order_hour_of_day           int64
user_avg_spacing          float64
streak                    float64
streak_-2.0                 uint8
streak_-3.0                 uint8
streak_-4.0                 uint8
streak_1.0                  uint8
streak_2.0                  uint8
streak_3.0                  uint8
streak_4.0                  uint8
streak_5.0                  uint8
streak_nan                  uint8
up_buy_streak               int64
up_n5_n_buys              float64
up_n5_buy_ratio           float64
up_atco_sum                 int64
up_atco_avg               float64
dtype: object

In [31]:
%%time
xgb = XGBClassifier(colsample_bytree=0.7,
                    min_child_weight=8,
                    n_estimators=460,
                    max_depth=8,
                    learning_rate=0.009,
                    eval_metric='logloss',
                    verbosity=3,
                    use_label_encoder =False)

xgb_fit = xgb.fit(X_train, y_train)

[05:47:14] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/gbm/gbtree.cc:146: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[05:47:14] DEBUG: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/gbm/gbtree.cc:154: Using tree method: 1
[05:47:14] DEBUG: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/gbm/gbtree.cc:119: Using updaters: grow_histmaker,prune
[05:47:14] ======== Monitor: TreePruner ========
[05:47:19] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 488 extra nodes, 0 pruned nodes, max_depth=8
[05:47:23] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 476 extra nodes, 0 pruned nodes, max_depth=8
[05:47:27] INFO: /home/conda/feedsto

[05:50:07] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 504 extra nodes, 0 pruned nodes, max_depth=8
[05:50:11] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 500 extra nodes, 0 pruned nodes, max_depth=8
[05:50:15] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 506 extra nodes, 0 pruned nodes, max_depth=8
[05:50:19] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 498 extra nodes, 0 pruned nodes, max_depth=8
[05:50:24] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 506 extra nodes, 0 pruned nodes, max_depth=8
[05:50:29] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree

[05:53:11] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 504 extra nodes, 0 pruned nodes, max_depth=8
[05:53:14] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[05:53:18] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 508 extra nodes, 0 pruned nodes, max_depth=8
[05:53:22] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 506 extra nodes, 0 pruned nodes, max_depth=8
[05:53:26] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 506 extra nodes, 0 pruned nodes, max_depth=8
[05:53:31] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree

[05:56:12] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 508 extra nodes, 0 pruned nodes, max_depth=8
[05:56:16] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 508 extra nodes, 0 pruned nodes, max_depth=8
[05:56:20] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 508 extra nodes, 0 pruned nodes, max_depth=8
[05:56:24] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 506 extra nodes, 0 pruned nodes, max_depth=8
[05:56:28] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 508 extra nodes, 0 pruned nodes, max_depth=8
[05:56:31] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree

[05:59:12] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 508 extra nodes, 0 pruned nodes, max_depth=8
[05:59:16] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 504 extra nodes, 0 pruned nodes, max_depth=8
[05:59:20] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 504 extra nodes, 0 pruned nodes, max_depth=8
[05:59:24] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[05:59:27] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 502 extra nodes, 0 pruned nodes, max_depth=8
[05:59:31] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree

...

[06:05:19] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 476 extra nodes, 0 pruned nodes, max_depth=8
[06:05:23] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 508 extra nodes, 0 pruned nodes, max_depth=8
[06:05:28] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 504 extra nodes, 0 pruned nodes, max_depth=8
[06:05:31] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 506 extra nodes, 0 pruned nodes, max_depth=8
[06:05:35] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:05:39] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree

[06:08:19] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 504 extra nodes, 0 pruned nodes, max_depth=8
[06:08:23] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 506 extra nodes, 0 pruned nodes, max_depth=8
[06:08:27] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 508 extra nodes, 0 pruned nodes, max_depth=8
[06:08:31] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 506 extra nodes, 0 pruned nodes, max_depth=8
[06:08:35] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 500 extra nodes, 0 pruned nodes, max_depth=8
[06:08:38] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree

[06:11:39] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 506 extra nodes, 0 pruned nodes, max_depth=8
[06:11:43] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 500 extra nodes, 0 pruned nodes, max_depth=8
[06:11:47] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 502 extra nodes, 0 pruned nodes, max_depth=8
[06:11:51] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 508 extra nodes, 0 pruned nodes, max_depth=8
[06:11:54] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 482 extra nodes, 0 pruned nodes, max_depth=8
[06:11:59] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree

[06:14:43] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 492 extra nodes, 0 pruned nodes, max_depth=8
[06:14:47] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 500 extra nodes, 0 pruned nodes, max_depth=8
[06:14:52] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 502 extra nodes, 0 pruned nodes, max_depth=8
[06:14:55] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 506 extra nodes, 0 pruned nodes, max_depth=8
[06:15:00] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 500 extra nodes, 0 pruned nodes, max_depth=8
[06:15:04] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree

[06:17:47] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 496 extra nodes, 0 pruned nodes, max_depth=8
[06:17:51] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 490 extra nodes, 0 pruned nodes, max_depth=8
[06:17:55] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 486 extra nodes, 0 pruned nodes, max_depth=8
[06:18:00] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 502 extra nodes, 0 pruned nodes, max_depth=8
[06:18:01] ======== Monitor: Learner ========
[06:18:01] Configure: 1e-06s, 1 calls @ 1us

[06:18:01] EvalOneIter: 0.003484s, 460 calls @ 3484us

[06:18:01] GetGradient: 36.1491s, 460 calls @ 36149099us

[06:18:01] PredictRaw: 49.4876s, 460 calls @ 49487575us

[06:18:01] Upda

In [32]:
with open(f"models/xgboost_fit_all.pickle", "wb") as pfile:
        pickle.dump(xgb_fit, pfile)

In [33]:
preds_all = xgb_fit.predict(X_test)

In [34]:
xgb_all_score = f1_score(y_test, preds_all)
xgb_all_score

0.2782669687117902

And now we'll do the same without `colsample_bytree` and `min_child_weight`. We'll call this model `xgb_fit_2`.

We'll then compare results.

In [35]:
%%time
xgb = XGBClassifier(n_estimators=460,
                    max_depth=8,
                    learning_rate=0.009,
                    eval_metric='logloss',
                    verbosity=3,
                    use_label_encoder =False)

xgb_fit_2 = xgb.fit(X_train, y_train)

[06:18:10] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/gbm/gbtree.cc:146: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[06:18:10] DEBUG: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/gbm/gbtree.cc:154: Using tree method: 1
[06:18:10] DEBUG: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/gbm/gbtree.cc:119: Using updaters: grow_histmaker,prune
[06:18:10] ======== Monitor: TreePruner ========
[06:18:17] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 504 extra nodes, 0 pruned nodes, max_depth=8
[06:18:23] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 504 extra nodes, 0 pruned nodes, max_depth=8
[06:18:29] INFO: /home/conda/feedsto

[06:22:36] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 508 extra nodes, 0 pruned nodes, max_depth=8
[06:22:42] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 508 extra nodes, 0 pruned nodes, max_depth=8
[06:22:48] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 506 extra nodes, 0 pruned nodes, max_depth=8
[06:22:54] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 508 extra nodes, 0 pruned nodes, max_depth=8
[06:23:00] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 508 extra nodes, 0 pruned nodes, max_depth=8
[06:23:06] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree

[06:27:22] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:27:29] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:27:35] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:27:41] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:27:47] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:27:53] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree

[06:32:08] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:32:14] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:32:20] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:32:27] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:32:33] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:32:39] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree

[06:36:55] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:37:01] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:37:07] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:37:14] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:37:20] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:37:26] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree

...

[06:46:33] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:46:39] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:46:46] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:46:52] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:46:58] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:47:04] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree

[06:51:27] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 506 extra nodes, 0 pruned nodes, max_depth=8
[06:51:34] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:51:40] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:51:46] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:51:52] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:51:58] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree

[06:56:18] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 508 extra nodes, 0 pruned nodes, max_depth=8
[06:56:24] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:56:30] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[06:56:36] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 508 extra nodes, 0 pruned nodes, max_depth=8
[06:56:42] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 508 extra nodes, 0 pruned nodes, max_depth=8
[06:56:49] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree

[07:01:08] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[07:01:14] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 508 extra nodes, 0 pruned nodes, max_depth=8
[07:01:21] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 508 extra nodes, 0 pruned nodes, max_depth=8
[07:01:27] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[07:01:34] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[07:01:40] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree

[07:06:01] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 506 extra nodes, 0 pruned nodes, max_depth=8
[07:06:07] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 508 extra nodes, 0 pruned nodes, max_depth=8
[07:06:13] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=8
[07:06:19] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 508 extra nodes, 0 pruned nodes, max_depth=8
[07:06:20] ======== Monitor: Learner ========
[07:06:20] Configure: 0.000497s, 1 calls @ 497us

[07:06:20] EvalOneIter: 0.003713s, 460 calls @ 3713us

[07:06:20] GetGradient: 35.9225s, 460 calls @ 35922473us

[07:06:20] PredictRaw: 46.6773s, 460 calls @ 46677299us

[07:06:20]

In [36]:
with open(f"models/xgboost_fit_base.pickle", "wb") as pfile:
        pickle.dump(xgb_fit_2, pfile)

In [37]:
preds_base = xgb_fit_2.predict(X_test)

In [38]:
xgb_base_score = f1_score(y_test, preds_base)
xgb_base_score

0.27804561248820814